<a href="https://colab.research.google.com/github/Pratibha65/NLPS/blob/main/NLPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

!pip install syllapy
import syllapy
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm.notebook import tqdm


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data = pd.read_csv('/content/Input.xlsx - Sheet1.csv')

In [ ]:

output_dir = 'extracted_articles'
os.makedirs(output_dir, exist_ok=True)

tags = ['h1', 'p', 'h3']

def extract_text_from_soup(soup):
    texts = []

    def extract_text_from_element(element):
        if element.name == 'ol' or element.name == 'ul':
            # Handle <ol> and <ul> tags
            for item in element.find_all('li', recursive=False):
                li_text = extract_text_from_element(item)
                if li_text:  # Check if li_text is not None or empty
                    texts.append(li_text)
                    # texts.append('\n')  # Add a newline after each <li>
        elif element.name == 'li':
            # Handle <li> elements
            li_text = []
            # if element.find_previous_sibling():
            #     li_text.append(element.find_previous_sibling().get_text(strip=True))  # Add text from <strong>
            li_text.append(element.get_text(strip=True))
            return "\n".join(li_text)
        # elif element.name in ['strong']:
        #     # Handle <strong> and other tags that might need special handling
        #     return element.get_text(strip=True)
        elif element.name in tags:
            text = element.get_text(strip=True)
            if text:  # Check if text is not None or empty
                texts.append(text)
        # else:
        #     text = element.get_text(strip=True)
        #     if text:  # Check if text is not None or empty
        #         texts.append(text)

        #Recursively handle children of the current element
        for child in element.find_all(recursive=False):
            child_text = extract_text_from_element(child)
            # if child_text:
            #     texts.append(child_text)

    extract_text_from_element(soup)
    return "\n\n".join(texts)

for index,row in data.iterrows():
  url_id = row['URL_ID']
  url = row['URL']
  try:
      response = requests.get(url)
      response.raise_for_status()  # Ensure we notice bad responses
      soup = BeautifulSoup(response.content, 'html.parser')
      title = soup.find('title').text
      soup = soup.find('div',class_='td-post-content tagdiv-type')
      full_text = extract_text_from_soup(soup)

      # Save the extracted text into a text file named with URL_ID
      file_name = os.path.join(output_dir, f'{url_id}.txt')
      with open(file_name, 'w', encoding='utf-8') as file:
          file.write(full_text)
      print(f"Article {url_id} processed successfully.")

  except requests.exceptions.RequestException as e:
      print(f"Failed to fetch URL {url}: {e}")
  except Exception as e:
      print(f"An error occurred while processing URL {url}: {e}")


Article bctech2011 processed successfully.
Article bctech2012 processed successfully.
Article bctech2013 processed successfully.
Article bctech2014 processed successfully.
Article bctech2015 processed successfully.
Article bctech2016 processed successfully.
Article bctech2017 processed successfully.
Article bctech2018 processed successfully.
Article bctech2019 processed successfully.
Article bctech2020 processed successfully.
Article bctech2021 processed successfully.
Article bctech2022 processed successfully.
Article bctech2023 processed successfully.
Article bctech2024 processed successfully.
Article bctech2025 processed successfully.
Article bctech2026 processed successfully.
Article bctech2027 processed successfully.
Article bctech2028 processed successfully.
Article bctech2029 processed successfully.
Article bctech2030 processed successfully.
Article bctech2031 processed successfully.
Article bctech2032 processed successfully.
Article bctech2033 processed successfully.
Article bct

In [ ]:
stop_words_folder = '/content/StopWords'
master_dict_folder = '/content/MasterDictionary'

In [ ]:
stop_words = set()
for file_name in os.listdir(stop_words_folder):
    file_path = os.path.join(stop_words_folder, file_name)
    with open(file_path, 'r', encoding='latin-1') as file:
        words = file.read().split()
        stop_words.update(words)

In [ ]:
stop_words_list = list(stop_words) # Convert the set to a list
stop_words_list.sort() # Sort the list
stopword_path = 'stopwords_list.xlsx'
stop_words_df = pd.DataFrame()
stop_words_df['Stop words'] = stop_words_list
stop_words_df.to_excel(stopword_path,index=False)

stop_words = set(stop_words_list) # Convert the list back to a set, if needed

In [ ]:
positive_words = set()
negative_words = set()

with open(os.path.join(master_dict_folder, 'positive-words.txt'), 'r', encoding='latin-1') as file:
    for word in file.read().split():
        if word not in stop_words:
            positive_words.add(word)

with open(os.path.join(master_dict_folder, 'negative-words.txt'), 'r', encoding='latin-1') as file:
    for word in file.read().split():
        if word not in stop_words:
            negative_words.add(word)

In [ ]:
def clean_text(text, stop_words):
    words = nltk.word_tokenize(text)
    cleaned_words = [word for word in words if word.lower() not in stop_words]
    return cleaned_words

In [ ]:
def calculate_sentiment_scores(tokens, positive_words, negative_words):
    positive_score = sum(1 for word in tokens if word in positive_words)
    negative_score = sum(1 for word in tokens if word in negative_words)
    total_words = len(tokens)

    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (total_words + 0.000001)

    return positive_score, negative_score, polarity_score, subjectivity_score

In [ ]:
def calculate_avg_sentence_length(text):
    sentences = sent_tokenize(text)
    total_sentences = len(sentences)
    total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
    avg_sentence_length = total_words / total_sentences if total_sentences > 0 else 0
    return avg_sentence_length

In [ ]:
def calculate_complex_word_percentage(tokens):
    complex_words = [word for word in tokens if syllapy.count(word) >= 3]
    total_words = len(tokens)
    percentage_complex_words = (len(complex_words) / total_words) * 100 if total_words > 0 else 0
    return percentage_complex_words

In [ ]:
def calculate_avg_words_per_sentence(text):
    sentences = sent_tokenize(text)
    total_sentences = len(sentences)
    total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
    avg_words_per_sentence = total_words / total_sentences if total_sentences > 0 else 0
    return avg_words_per_sentence

In [ ]:
def calculate_word_count(tokens):
    return len(tokens)

In [ ]:
def count_syllables(word_list):
    total_syllables = 0
    for word in word_list:
      word = word.lower()
      vowels = "aeiou"
      syllable_count = 0
      prev_char_was_vowel = False

      for index, char in enumerate(word):
          if char in vowels:
              if not prev_char_was_vowel:
                  syllable_count += 1
              prev_char_was_vowel = True
          else:
              prev_char_was_vowel = False

      # Handle exceptions
      if word.endswith("es") and len(word) > 2 and not word.endswith("ses"):
          syllable_count -= 1
      elif word.endswith("ed") and len(word) > 2 and not word.endswith("ted") and not word.endswith("ded"):
          syllable_count -= 1

      if syllable_count == 0:
          syllable_count = 1

      total_syllables += syllable_count

    return total_syllables

In [ ]:
def calculate_complex_word_count(tokens):
    complex_words = [word for word in tokens if syllapy.count(word) >= 3]
    return len(complex_words)

In [ ]:
def calculate_personal_pronoun_count(tokens):
    personal_pronouns = {'I','me','you','mine','he','she','it', 'we','they','them','him','her','his','hers','its','theirs','our','your', 'my', 'ours', 'us'}
    return sum(1 for word in tokens if word.lower() in personal_pronouns)

In [ ]:
def calculate_avg_word_length(tokens):
    total_length = sum(len(word) for word in tokens)
    total_words = len(tokens)
    avg_word_length = total_length / total_words if total_words > 0 else 0
    return avg_word_length

In [ ]:
positive_scores = []
negative_scores = []
polarity_scores = []
subjectivity_scores = []
avg_sentence_lengths = []
complex_word_percentages = []
avg_words_per_sentence_list = []
fog_index = []
word_counts = []
syllables_per_word_list = []
complex_word_counts = []
personal_pronoun_counts = []
avg_word_lengths = []

for index, row in tqdm(data.iterrows(), total=data.shape[0]):
    url_id = row['URL_ID']
    file_path = os.path.join(output_dir, f'{url_id}.txt')

    if os.path.exists(file_path):
        # Read the text content of the file
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        cleaned_tokens = clean_text(text, stop_words)
        positive_score, negative_score, polarity_score, subjectivity_score = calculate_sentiment_scores(cleaned_tokens, positive_words, negative_words)
        avg_sentence_length = calculate_avg_sentence_length(text)
        complex_word_percentage = calculate_complex_word_percentage(cleaned_tokens)
        avg_words_per_sentence = calculate_avg_words_per_sentence(text)
        word_count = calculate_word_count(cleaned_tokens)
        syllables_per_word = (count_syllables(cleaned_tokens)/word_count)
        complex_word_count = calculate_complex_word_count(cleaned_tokens)
        personal_pronoun_count = calculate_personal_pronoun_count(cleaned_tokens)
        avg_word_length = calculate_avg_word_length(cleaned_tokens)

        positive_scores.append(positive_score)
        negative_scores.append(negative_score)
        polarity_scores.append(polarity_score)
        subjectivity_scores.append(subjectivity_score)
        avg_sentence_lengths.append(avg_sentence_length)
        complex_word_percentages.append(complex_word_percentage)
        avg_words_per_sentence_list.append(avg_words_per_sentence)
        fog_index.append(0.4*(avg_sentence_length+complex_word_percentage))
        word_counts.append(word_count)
        syllables_per_word_list.append(syllables_per_word)
        complex_word_counts.append(complex_word_count)
        personal_pronoun_counts.append(personal_pronoun_count)
        avg_word_lengths.append(avg_word_length)
    else:
        positive_scores.append(None)
        negative_scores.append(None)
        polarity_scores.append(None)
        subjectivity_scores.append(None)
        avg_sentence_lengths.append(None)
        complex_word_percentages.append(None)
        avg_words_per_sentence_list.append(None)
        fog_index.append(None)
        word_counts.append(None)
        syllables_per_word_list.append(None)
        complex_word_counts.append(None)
        personal_pronoun_counts.append(None)
        avg_word_lengths.append(None)

data['POSITIVE SCORE'] = positive_scores
data['NEGATIVE SCORE'] = negative_scores
data['POLARITY SCORE'] = polarity_scores
data['SUBJECTIVITY SCORE'] = subjectivity_scores
data['AVG SENTENCE LENGTH'] = avg_sentence_lengths
data['PERCENTAGE OF COMPLEX WORDS'] = complex_word_percentages
data['FOG INDEX'] = fog_index
data['AVG NUMBER OF WORDS PER SENTENCE'] = avg_words_per_sentence_list
data['COMPLEX WORD COUNT'] = complex_word_counts
data['WORD COUNT'] = word_counts
data['SYLLABLE PER WORD'] = syllables_per_word_list
data['PERSONAL PRONOUNS'] = personal_pronoun_counts
data['AVG WORD LENGTH'] = avg_word_lengths


  0%|          | 0/147 [00:00<?, ?it/s]

In [ ]:
data['PERSONAL PRONOUNS'].value_counts()


PERSONAL PRONOUNS
0    147
Name: count, dtype: int64

In [ ]:
output_excel_path = 'updated_input_with_sentiments.xlsx'
data.to_excel(output_excel_path, index=False)

print("Sentiment scores have been added to the DataFrame and saved to the new Excel file.")

Sentiment scores have been added to the DataFrame and saved to the new Excel file.


In [ ]:
output_csv_path='output_csv_file.csv'

data.to_csv(output_csv_path, index=False)